In [1]:
import pandas as pd
import json

In [5]:
rating = pd.read_csv('네이버맵음식점평점_최종.csv')
num = pd.DataFrame(rating.value_counts('name'))
star = rating.pivot_table('rating',index='name',aggfunc='mean')

#### 리뷰 한문장으로 만들고 평균 평점, 리뷰 개수 생성 함수

In [113]:
# df = pd.read_json('이종민산출물/review_test_1947.json')
rating = pd.read_csv('네이버맵음식점평점_최종.csv')

def make_review_data(rating,category,percentile=0.6):

#     review = [' '.join(map(str,list(df.T.loc[i].dropna()))).replace('\n',' ') for i in df.T.index]
#     df2 = pd.DataFrame(index=df.T.index)
#     df2['review']=review
    
    num = pd.DataFrame(rating.value_counts('name'))
    star = rating.pivot_table('rating',index='name',aggfunc='mean')
    
#     df3 = df2.merge(star,left_index=True,right_index=True,how='left').merge(num,left_index=True,right_index=True,how='left')
    
    df3 = star.merge(num,left_index=True,right_index=True,how='left')
    df3.columns = ['rating','num']
#     df3['category']=list((rating[rating['name']==df_review.index[0]])['category'])[0]
    
    df3= df3.fillna(0)

    
### 가중 평점 계산 후 상위 10개 가게 반환 함수
    
    m = df3['num'].quantile(percentile)
    C = df3['rating'].mean()

    # 기존 평점을 새로운 가중 평점으로 변경하는 함수 생성
    # - vote_count, vote_average, m, C 값을 적용해서
    # - 레코드별 가중 평점 반환

    def weighted_vote_average(record):
        v = record['num']
        R = record['rating']

        return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

    df3['weighted_rating'] = df3.apply(weighted_vote_average, axis=1) 
    a=pd.DataFrame(rating.groupby(['name'])['category'].first())
    df3=df3.merge(a,left_index=True,right_index=True)
    df3=df3[df3['category']==category]
    
    return df3[['rating','weighted_rating','num','category']].sort_values('weighted_rating',
                                                                          ascending=False)[:10]

In [115]:
make_review_data(rating,'서귀포시')

,rating,weighted_rating,num,category
name,,,,
중문색달통갈치,4.903427,4.896247,1926,서귀포시
중문흑돼지천국,4.895531,4.887343,1656,서귀포시
칼초네화덕피자,4.891478,4.882546,1502,서귀포시
포트그릴스테이크하우스,4.907843,4.881592,510,서귀포시
찰리스,4.880623,4.865730,867,서귀포시
썬라이즈135,4.872483,4.851533,596,서귀포시
오또도넛,4.868805,4.850642,686,서귀포시
짱실장돈까스,4.978261,4.844752,92,서귀포시
탐나돈정육점식당,4.844340,4.836235,1484,서귀포시
